In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta

import os
os.chdir('/Users/juliayu001/code/alecmatt5/nba_betting_analysis/backend/logic')
from today_games_preprocessor import preprocess_advanced, get_basic_boxscores, roll
os.chdir('/Users/juliayu001/code/alecmatt5/nba_betting_analysis/backend')

In [ ]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
team_names = [team['full_name'] for team in nba_teams]
team_names.sort()
team_ids = [team['id'] for team in nba_teams]

In [2]:
preproc_part1, X_features = preprocess_advanced('boxscores_advanced_team_all.pkl',
                                        roll_methods=['mean'],
                                        ohe=True,
                                        scaled=False)

/Users/juliayu001/code/alecmatt5/nba_betting_analysis/backend/logic/today_games_preprocessor.py:55: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['TEAM_ABBREVIATION'], dtype='object')
  return team.rolling(roll_number, closed='left').mean()
/Users/juliayu001/.pyenv/versions/nba_betting_analysis/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
df = pd.read_pickle('data/pkl/boxscores_advanced_team_all.pkl')

In [ ]:
df.head(20)

In [3]:
preproc_part1

,GAME_ID,TEAM_ID_h,TEAM_NAME_h,TEAM_ABBREVIATION_h,GAME_DATE,HOME_TEAM_h,PLUS_MINUS,OFF_RATING_Roll_mean_h,DEF_RATING_Roll_mean_h,NET_RATING_Roll_mean_h,...,OKC_a,ORL_a,PHI_a,PHX_a,POR_a,SAC_a,SAS_a,TOR_a,UTA_a,WAS_a
0,22201019,1610612747,Lakers,LAL,2023-03-12,1,-4.0,110.400,105.825,4.600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22201018,1610612759,Spurs,SAS,2023-03-12,1,-12.0,111.875,117.900,-6.000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22201017,1610612740,Pelicans,NOP,2023-03-12,1,17.0,106.725,114.375,-7.650,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,22201016,1610612755,76ers,PHI,2023-03-12,1,19.0,131.350,123.375,8.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,22201015,1610612766,Hornets,CHA,2023-03-12,1,-6.0,107.050,109.275,-2.200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,22200709,1610612745,Rockets,HOU,2023-01-23,1,5.0,111.175,122.025,-10.850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307,22200705,1610612757,Trail Blazers,POR,2023-01-22,1,-9.0,123.725,120.225,3.450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,22200704,1610612744,Warriors,GSW,2023-01-22,1,-4.0,114.550,114.950,-0.425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,22200701,1610612742,Mavericks,DAL,2023-01-22,1,-14.0,124.450,128.050,-3.575,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Today's games

In [53]:
today = datetime.now().strftime('%Y-%m-%d')

In [68]:
df1 = team_df[['home_team_id', 'home_team', 'game_id']]
df1.rename(columns={'game_id': 'GAME_ID', 'home_team': 'TEAM_ABBREVIATION', 'home_team_id': 'TEAM_ID'}, inplace=True)
df1['GAME_DATE'] = today
df1['HOME_TEAM'] = 1
df1['PLUS_MINUS'] = 0

/var/folders/ry/bbrg927549539gbw_r9_cqyc0000gn/T/ipykernel_15008/1509773145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'game_id': 'GAME_ID', 'home_team': 'TEAM_ABBREVIATION', 'home_team_id': 'TEAM_ID'}, inplace=True)
/var/folders/ry/bbrg927549539gbw_r9_cqyc0000gn/T/ipykernel_15008/1509773145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['GAME_DATE'] = today


,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,1610612766,CHA,0022201027,2023-03-14,1,0
1,1610612764,WAS,0022201028,2023-03-14,1,0
2,1610612761,TOR,0022201029,2023-03-14,1,0
3,1610612740,NOP,0022201030,2023-03-14,1,0
4,1610612760,OKC,0022201031,2023-03-14,1,0
5,1610612759,SAS,0022201032,2023-03-14,1,0
6,1610612756,PHX,0022201033,2023-03-14,1,0
7,1610612757,POR,0022201034,2023-03-14,1,0


In [67]:
df2 = team_df[['away_team_id', 'away_team', 'game_id']]
df2.rename(columns={'game_id': 'GAME_ID', 'away_team': 'TEAM_ABBREVIATION', 'away_team_id': 'TEAM_ID'}, inplace=True)
df2['GAME_DATE'] = today
df2['HOME_TEAM'] = 0
df2['PLUS_MINUS'] = 0

/var/folders/ry/bbrg927549539gbw_r9_cqyc0000gn/T/ipykernel_15008/3658351368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'game_id': 'GAME_ID', 'away_team': 'TEAM_ABBREVIATION', 'away_team_id': 'TEAM_ID'}, inplace=True)
/var/folders/ry/bbrg927549539gbw_r9_cqyc0000gn/T/ipykernel_15008/3658351368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['GAME_DATE'] = today


,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,1610612739,CLE,0022201027,2023-03-14,0,0
1,1610612765,DET,0022201028,2023-03-14,0,0
2,1610612743,DEN,0022201029,2023-03-14,0,0
3,1610612747,LAL,0022201030,2023-03-14,0,0
4,1610612751,BKN,0022201031,2023-03-14,0,0
5,1610612753,ORL,0022201032,2023-03-14,0,0
6,1610612749,MIL,0022201033,2023-03-14,0,0
7,1610612752,NYK,0022201034,2023-03-14,0,0


In [78]:
games_today_df = pd.concat([df1, df2], ignore_index=True, sort=False)

In [84]:
advanced_today_df = games_today_df
advanced_today_df

columns = ['TEAM_NAME', 'TEAM_CITY',
'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE',
'PACE_PER40', 'POSS', 'PIE']

for column in columns:
    advanced_today_df[column] = 0

advanced_today_df = advanced_today_df.reindex(columns=['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
                                                       'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
                                                       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
                                                       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
                                                       'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE',
                                                       'PACE_PER40', 'POSS', 'PIE'])


# Debug

In [4]:
from datetime import datetime, timedelta
date = datetime.now() - timedelta(days=50)
date_str = date.strftime('%Y-%m-%d')

In [5]:
date_str

'2023-01-23'

In [6]:
basic = get_basic_boxscores(date=date_str)
games_df = basic[['TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']].copy()

In [25]:
games_df

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,1610612740,NOP,22201017,2023-03-12,1,17.0
1,1610612755,PHI,22201016,2023-03-12,1,19.0
2,1610612764,WAS,22201016,2023-03-12,0,-19.0
3,1610612766,CHA,22201015,2023-03-12,1,-6.0
4,1610612739,CLE,22201015,2023-03-12,0,6.0
...,...,...,...,...,...,...
609,1610612764,WAS,22200717,2023-01-24,0,1.0
610,1610612752,NYK,22200715,2023-01-24,1,2.0
611,1610612754,IND,22200713,2023-01-24,1,6.0
612,1610612756,PHX,22200718,2023-01-24,1,31.0


In [ ]:
games_df.head(600)['TEAM_ABBREVIATION'].value_counts()

In [7]:
#get advanced boxscore data from pickle
advanced = pd.read_pickle('data/pkl/boxscores_advanced_team_all.pkl')

In [ ]:
advanced = advanced.head(600)

In [8]:
#drop unecessary columns
columns_to_drop = ['TEAM_CITY', 'MIN', 'E_OFF_RATING', 'E_DEF_RATING',
               'E_NET_RATING', 'AST_RATIO', 'E_TM_TOV_PCT', 'USG_PCT',
               'E_USG_PCT', 'E_PACE', 'PACE_PER40', 'PIE']
advanced = advanced.drop(columns=columns_to_drop)

In [9]:
#change game_id type to match between the 2 data frames
advanced['GAME_ID'] = advanced['GAME_ID'].astype('int32')

In [10]:
#merge the needed columns from basic to advanced
advanced = advanced.merge(games_df.drop(columns=['TEAM_ID']), on=['GAME_ID', 'TEAM_ABBREVIATION'])

In [11]:
advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,22200963,1610612754,Pacers,IND,133.0,131.2,1.8,0.563,3.0,0.319,0.742,0.487,9.6,0.596,0.619,93.5,94,2023-03-05,0,3.0
1,22200963,1610612741,Bulls,CHI,131.2,133.0,-1.8,0.4,1.2,0.258,0.681,0.513,16.1,0.676,0.718,93.5,93,2023-03-05,1,-3.0
2,22200962,1610612756,Suns,PHX,134.0,128.6,5.4,0.551,3.0,0.317,0.829,0.573,9.3,0.618,0.65,97.5,97,2023-03-05,0,4.0
3,22200962,1610612742,Mavericks,DAL,128.6,134.0,-5.4,0.395,1.5,0.171,0.683,0.427,10.2,0.591,0.672,97.5,98,2023-03-05,1,-4.0
4,22200964,1610612747,Lakers,LAL,114.1,106.1,8.1,0.69,3.22,0.309,0.754,0.536,9.1,0.5,0.54,99.0,99,2023-03-05,1,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,22201005,1610612766,Hornets,CHA,113.3,120.2,-6.9,0.6,3.43,0.254,0.559,0.407,7.1,0.495,0.531,98.5,98,2023-03-11,1,-8.0
646,22200769,1610612764,Wizards,WAS,128.0,127.2,0.8,0.574,1.8,0.371,0.659,0.532,16.1,0.654,0.672,92.5,93,2023-03-07,0,2.0
647,22200769,1610612765,Pistons,DET,127.2,128.0,-0.8,0.674,2.42,0.341,0.629,0.468,13.0,0.589,0.624,92.5,92,2023-03-07,1,-2.0
648,22200992,1610612765,Pistons,DET,103.0,111.9,-8.9,0.5,1.05,0.353,0.694,0.52,19.0,0.489,0.527,100.5,100,2023-03-09,1,-10.0


In [12]:
advanced = advanced.sort_values(by=['GAME_DATE', 'GAME_ID', 'HOME_TEAM'], ascending=False).reset_index(drop=True)

In [13]:
advanced = advanced.drop_duplicates()

In [14]:
#drop rows that only have 1 team for the game id
value_counts = advanced['GAME_ID'].value_counts()
unique_values = value_counts[value_counts == 1].index.tolist()
advanced = advanced[~advanced['GAME_ID'].isin(unique_values)]
advanced = advanced.reset_index(drop=True)

In [15]:
advanced_desc = advanced.sort_values(by=['GAME_DATE'], ascending=True).copy()
advanced_desc

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
613,22200713,1610612741,Bulls,CHI,110.0,116.0,-6.0,0.442,1.19,0.327,0.644,0.5,16.0,0.517,0.556,100.0,100,2023-01-24,0,-6.0
600,22200719,1610612747,Lakers,LAL,119.8,138.5,-18.8,0.5,3.0,0.293,0.714,0.452,7.3,0.526,0.545,96.0,96,2023-01-24,1,-18.0
601,22200719,1610612746,Clippers,LAC,138.5,119.8,18.8,0.49,1.67,0.286,0.707,0.548,15.6,0.72,0.734,96.0,96,2023-01-24,0,18.0
602,22200718,1610612756,Suns,PHX,124.3,95.1,29.2,0.809,2.11,0.362,0.75,0.579,17.5,0.617,0.642,102.5,103,2023-01-24,1,31.0
603,22200718,1610612766,Hornets,CHA,95.1,124.3,-29.2,0.487,1.36,0.25,0.638,0.421,13.7,0.438,0.467,102.5,102,2023-01-24,0,-31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,22201015,1610612766,Hornets,CHA,106.9,115.2,-8.2,0.61,1.32,0.262,0.735,0.516,18.8,0.575,0.602,100.0,101,2023-03-12,1,-6.0
9,22201015,1610612739,Cavaliers,CLE,115.2,106.9,8.2,0.439,1.8,0.265,0.738,0.484,10.1,0.511,0.559,100.0,99,2023-03-12,0,6.0
11,22201014,1610612751,Nets,BKN,129.8,126.3,3.5,0.762,3.2,0.186,0.521,0.363,10.6,0.601,0.645,94.5,94,2023-03-12,0,2.0
6,22201016,1610612755,76ers,PHI,121.7,102.2,19.5,0.707,3.22,0.275,0.75,0.543,9.8,0.581,0.621,91.5,92,2023-03-12,1,19.0


In [16]:
#define features to engineer
non_eng_features = ['TEAM_ABBREVIATION', 'GAME_ID', 'TEAM_ID', 'TEAM_NAME',
                    'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']
eng_features = advanced_desc.drop(columns=non_eng_features).columns.tolist()

In [17]:
roll_methods=['mean']

In [18]:
#caluculate rolling metrics
if 'mean' in roll_methods:
    df_temp = roll(df = advanced_desc, roll_number=4, procedure='mean', selected_columns=eng_features)
    advanced = advanced.merge(df_temp, left_index=True, right_index=True)
if 'median' in roll_methods:
    df_temp = roll(df = advanced_desc, roll_number=4, procedure='median', selected_columns=eng_features)
    advanced = advanced.merge(df_temp, left_index=True, right_index=True)
if 'std' in roll_methods:
    df_temp = roll(df = advanced_desc, roll_number=4, procedure='std', selected_columns=eng_features)
    advanced = advanced.merge(df_temp, left_index=True, right_index=True)

/Users/juliayu001/code/alecmatt5/nba_betting_analysis/backend/logic/today_games_preprocessor.py:55: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['TEAM_ABBREVIATION'], dtype='object')
  return team.rolling(roll_number, closed='left').mean()


In [19]:
advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,OREB_PCT,...,AST_PCT_Roll_mean,AST_TOV_Roll_mean,OREB_PCT_Roll_mean,DREB_PCT_Roll_mean,REB_PCT_Roll_mean,TM_TOV_PCT_Roll_mean,EFG_PCT_Roll_mean,TS_PCT_Roll_mean,PACE_Roll_mean,POSS_Roll_mean
0,22201019,1610612747,Lakers,LAL,114.9,117.9,-3.0,0.558,2.18,0.286,...,0.65250,1.9700,0.26175,0.77250,0.52975,14.725,0.52725,0.57450,101.7500,101.75
1,22201019,1610612752,Knicks,NYK,117.9,114.9,3.0,0.429,1.64,0.319,...,0.53900,1.7950,0.31875,0.73800,0.51300,12.550,0.48425,0.52575,96.4175,101.25
2,22201018,1610612759,Spurs,SAS,89.1,100.0,-10.9,0.727,1.26,0.241,...,0.61600,1.6450,0.30900,0.75275,0.51425,16.125,0.54325,0.56975,102.3750,102.25
3,22201018,1610612760,Thunder,OKC,100.0,89.1,10.9,0.722,2.89,0.25,...,0.56525,1.6050,0.29175,0.70975,0.50150,14.075,0.54675,0.59850,102.7500,102.75
4,22201017,1610612740,Pelicans,NOP,121.0,103.8,17.2,0.814,2.33,0.213,...,0.67500,1.6325,0.21675,0.78875,0.50350,16.875,0.54750,0.58550,97.6250,97.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,22200715,1610612739,Cavaliers,CLE,110.8,111.7,-0.9,0.725,2.42,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,22200714,1610612748,Heat,MIA,103.2,101.1,2.1,0.618,2.63,0.339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,22200714,1610612738,Celtics,BOS,101.1,103.2,-2.1,0.656,1.24,0.178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
612,22200713,1610612754,Pacers,IND,116.0,110.0,6.0,0.571,3.0,0.356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#drop original columns to prevent data leakage
drop_columns = ['OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'OREB_PCT', 'DREB_PCT',
    'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'PACE', 'POSS']
advanced.drop(columns=drop_columns, inplace=True)

In [21]:
advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,GAME_DATE,HOME_TEAM,PLUS_MINUS,OFF_RATING_Roll_mean,DEF_RATING_Roll_mean,NET_RATING_Roll_mean,AST_PCT_Roll_mean,AST_TOV_Roll_mean,OREB_PCT_Roll_mean,DREB_PCT_Roll_mean,REB_PCT_Roll_mean,TM_TOV_PCT_Roll_mean,EFG_PCT_Roll_mean,TS_PCT_Roll_mean,PACE_Roll_mean,POSS_Roll_mean
0,22201019,1610612747,Lakers,LAL,2023-03-12,1,-4.0,110.400,105.825,4.600,0.65250,1.9700,0.26175,0.77250,0.52975,14.725,0.52725,0.57450,101.7500,101.75
1,22201019,1610612752,Knicks,NYK,2023-03-12,0,4.0,110.375,115.650,-5.300,0.53900,1.7950,0.31875,0.73800,0.51300,12.550,0.48425,0.52575,96.4175,101.25
2,22201018,1610612759,Spurs,SAS,2023-03-12,1,-12.0,111.875,117.900,-6.000,0.61600,1.6450,0.30900,0.75275,0.51425,16.125,0.54325,0.56975,102.3750,102.25
3,22201018,1610612760,Thunder,OKC,2023-03-12,0,12.0,115.800,115.500,0.275,0.56525,1.6050,0.29175,0.70975,0.50150,14.075,0.54675,0.59850,102.7500,102.75
4,22201017,1610612740,Pelicans,NOP,2023-03-12,1,17.0,106.725,114.375,-7.650,0.67500,1.6325,0.21675,0.78875,0.50350,16.875,0.54750,0.58550,97.6250,97.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,22200715,1610612739,Cavaliers,CLE,2023-01-24,0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,22200714,1610612748,Heat,MIA,2023-01-24,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,22200714,1610612738,Celtics,BOS,2023-01-24,0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
612,22200713,1610612754,Pacers,IND,2023-01-24,1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#split data frame between the home teams and the away teams
advanced = advanced.sort_values(by=['GAME_DATE', 'GAME_ID', 'HOME_TEAM'], ascending=False).reset_index(drop=True)
adv_home = advanced.iloc[::2].copy()
adv_away = advanced.iloc[1::2].copy()

In [23]:
advanced.head(30)

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,GAME_DATE,HOME_TEAM,PLUS_MINUS,OFF_RATING_Roll_mean,DEF_RATING_Roll_mean,NET_RATING_Roll_mean,AST_PCT_Roll_mean,AST_TOV_Roll_mean,OREB_PCT_Roll_mean,DREB_PCT_Roll_mean,REB_PCT_Roll_mean,TM_TOV_PCT_Roll_mean,EFG_PCT_Roll_mean,TS_PCT_Roll_mean,PACE_Roll_mean,POSS_Roll_mean
0,22201019,1610612747,Lakers,LAL,2023-03-12,1,-4.0,110.400,105.825,4.600000e+00,0.65250,1.9700,0.26175,0.77250,0.52975,14.725,0.52725,0.57450,101.7500,101.75
1,22201019,1610612752,Knicks,NYK,2023-03-12,0,4.0,110.375,115.650,-5.300000e+00,0.53900,1.7950,0.31875,0.73800,0.51300,12.550,0.48425,0.52575,96.4175,101.25
2,22201018,1610612759,Spurs,SAS,2023-03-12,1,-12.0,111.875,117.900,-6.000000e+00,0.61600,1.6450,0.30900,0.75275,0.51425,16.125,0.54325,0.56975,102.3750,102.25
3,22201018,1610612760,Thunder,OKC,2023-03-12,0,12.0,115.800,115.500,2.750000e-01,0.56525,1.6050,0.29175,0.70975,0.50150,14.075,0.54675,0.59850,102.7500,102.75
4,22201017,1610612740,Pelicans,NOP,2023-03-12,1,17.0,106.725,114.375,-7.650000e+00,0.67500,1.6325,0.21675,0.78875,0.50350,16.875,0.54750,0.58550,97.6250,97.50
5,22201017,1610612757,Trail Blazers,POR,2023-03-12,0,-17.0,112.125,115.475,-3.375000e+00,0.65000,1.9850,0.20675,0.77625,0.49350,13.275,0.55025,0.59475,99.2500,99.25
6,22201016,1610612755,76ers,PHI,2023-03-12,1,19.0,131.350,123.375,8.000000e+00,0.61200,2.7225,0.26850,0.77200,0.53375,11.175,0.61500,0.66925,98.2500,98.25
7,22201016,1610612764,Wizards,WAS,2023-03-12,0,-19.0,115.425,118.625,-3.200000e+00,0.59550,1.9300,0.29800,0.75050,0.52900,14.625,0.57100,0.59575,99.2500,99.25
8,22201015,1610612766,Hornets,CHA,2023-03-12,1,-6.0,107.050,109.275,-2.200000e+00,0.57900,2.3075,0.24275,0.69875,0.47750,10.625,0.50675,0.53625,98.5000,98.75
9,22201015,1610612739,Cavaliers,CLE,2023-03-12,0,6.0,112.575,105.350,7.175000e+00,0.64100,2.0050,0.25025,0.68100,0.46225,13.775,0.54875,0.58000,97.8400,100.25


# Basic boxscores

In [ ]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import scoreboard

# Get yesterday's date
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(days=1)
yesterday_str = yesterday.strftime('%m/%d/%Y')

scoreboard_ = scoreboard.Scoreboard(game_date=yesterday_str, league_id='00', day_offset=0)
games = scoreboard_.game_header.get_data_frame()
if not games.empty:
    game_ids = list(games['GAME_ID'])
    print(team['full_name'], game_ids)

# Advanced boxscores

The command above/below gets the advanced boxscore from a given game id

boxscores_advanced.get_data_frames()[0] returns the advanced boxscore broken up per player

boxscores_advanced.get_data_frames()[1] returns the advanced boxscore broken up per team

In [ ]:
from nba_api.stats.endpoints import boxscoreadvancedv2

boxscores_advanced_player = None
boxscores_advanced_team = None

for game_id in game_ids:
    if boxscores_advanced_team is None:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = gamefinder.get_data_frames()[1]
        boxscores_advanced_player = gamefinder.get_data_frames()[0]
    else:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = pd.concat([boxscores_advanced_team, gamefinder.get_data_frames()[1]])
        boxscores_advanced_player = pd.concat([boxscores_advanced_player, gamefinder.get_data_frames()[0]])

In [ ]:
boxscores_advanced_team.columns

In [ ]:
# boxscores_advanced_team_yesterday.to_pickle('boxscores_advanced_team_part2.pkl')

In [ ]:
# boxscores_advanced_player_yesterday.to_pickle('boxscores_advanced_player_part2.pkl')

In [ ]:
df = pd.read_pickle('../data/pkl/raw_games_5yrs.pkl')

In [ ]:
df['GAME_DATE']